In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from MyKDTree import KDTree

In [16]:
tree = KDTree()

tree.add_point(54, 22)
tree.add_point(34, 13)
tree.add_point(26, 5)
tree.add_point(62, 42)
tree.add_point(45, 16)
tree.add_point(36, 12)
tree.add_point(53, 19)
tree.add_point(47, 25)
tree.add_point(51, 30)

# build the tree
tree.build(split_between=True)

# print three
tree.print()

# draw split
matplotlib.rcParams['figure.figsize'] = [10, 5]
tree.draw_split()

# draw tree
matplotlib.rcParams['figure.figsize'] = [12, 6]
tree.draw_tree()

AttributeError: 'Point' object has no attribute 'depth'